## 1. The existing data set puts 18 characteristic data of each company in one row for 20 years. 

In [1]:
# The data set has a total of 360+5=365 columns and a total of 8971 rows (different companies)

In [2]:
import pandas as pd
file = "new_american_bankruptcy_dataset.csv"
new_df = pd.read_csv(file)
new_df.head()

,company_name,X1_1999,X2_1999,X3_1999,X4_1999,X5_1999,X6_1999,X7_1999,X8_1999,X9_1999,...,X13_2018,X14_2018,X15_2018,X16_2018,X17_2018,X18_2018,status_label,Division,MajorGroup,last_year
0,C_1,511267.0,740998.00,833107.000,180447.0,18373.000,70658.0,89031.00,191226.00,336.018,...,NaN,NaN,NaN,NaN,NaN,NaN,alive,D,37,2017.0
1,C_2,1029438.0,1672529.00,930142.000,11024.0,102.090,311649.0,413739.00,996805.00,243882.000,...,NaN,NaN,NaN,NaN,NaN,NaN,alive,D,36,2010.0
2,C_3,9757.0,13986.00,19796.000,5974.0,667.000,-932.0,-265.00,9574.00,5494.000,...,NaN,NaN,NaN,NaN,NaN,NaN,alive,D,38,2008.0
3,C_4,381872.0,1160266.00,366.683,591784.0,25.633,97527.0,123.16,365.76,161033.000,...,NaN,NaN,NaN,NaN,NaN,NaN,alive,D,28,2007.0
4,C_5,28957.0,42.21,79567.000,591.0,2024.000,1849.0,3873.00,27743.00,10947.000,...,NaN,NaN,NaN,NaN,NaN,NaN,alive,D,35,1999.0


In [3]:
# Count the number of rows with missing values
rows_with_missing_values = new_df.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 1172
Number of rows with missing values： 7799


## 2. In the adjusted data set, the year expression is changed to the last 1-6 years, and the data set is reduced to only the last 6 years instead of 20 years

#### 2.1 Only accounting data

In [4]:
# (X1-18 of the last 6 years, 108 columns, the data set has a total of 113 columns).
#A total of 8971 rows (different companies), but the number of rows without missing values has increased (because the year is reduced to 6 years)

In [5]:
import pandas as pd
import numpy as np
import warnings

# Ignore warning
warnings.filterwarnings('ignore')

# Create a new DataFrame to store the transformed dataset
new_df_transformed = new_df.copy()

# Define a function to do column renaming
def rename_columns(df, years_to_keep):
    new_columns = []
    for i in range(1, 19):
        for j in range(1, years_to_keep + 1):
            column_name_new = f'X{i}_last{j}year'
            new_columns.append(column_name_new)
            df[column_name_new] = np.nan
    return df, new_columns

# Add new column
new_df_transformed, new_columns = rename_columns(new_df_transformed, 6)

# Iterate through each company and fill the columns
for index, row in new_df_transformed.iterrows():
    company_last_year = row['last_year']
    if not np.isnan(company_last_year):
        for j in range(1, 7):
            for i in range(1, 19):
                column_name_old = f'X{i}_{int(company_last_year) - j + 1}'
                column_name_new = f'X{i}_last{j}year'
                if column_name_old in new_df_transformed.columns:
                    new_df_transformed.at[index, column_name_new] = row[column_name_old]
                else:
                    # Fill in missing values for non-existent year data
                    new_df_transformed.at[index, column_name_new] = np.nan

# Remove the original year column
for year in range(1999, 2019):
    for i in range(1, 19):
        column_name_old = f'X{i}_{year}'
        if column_name_old in new_df_transformed.columns:
            new_df_transformed.drop(columns=[column_name_old], inplace=True)

# Print the first 5 rows of the new DataFrame
print(new_df_transformed.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X1_last4year  X1_last5year  ...  X17_last3year  \
0         888.5         873.1         954.1      1116.900  ...         1662.6   
1         900.2        1077.4        1008.2       942.700  ...         1456.4   
2       13454.0       13582.0        7726.0      5807.000  ...        24998.0   
3      353541.0     1037047.0      672072.0       692.991  ...       553617.0   
4           NaN           NaN           NaN           NaN  ...            NaN   

   X17_last4year  X17_last5year  X17_last6year  X1

In [6]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_transformed.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 4978
Number of rows with missing values： 3993


In [7]:
#
unique_divisions = new_df_transformed['Division'].nunique()
print("Number of unique values in 'Division' column:", unique_divisions)


Number of unique values in 'Division' column: 10


#### 2.2 Add the year-on-year change rate for the last 5 years

In [8]:
# In the current data set, the year-on-year change rate is added (X1-18 in the last 6 years, 108 columns + the year-on-year change rate in the last 5 years, 90 columns, the data set has a total of 203 columns), and the number of rows and missing values remain unchanged.


In [9]:
import pandas as pd

# Create a new DataFrame to store the modified data
modified_df = new_df_transformed.copy()

# Create a new column to store the year-over-year growth rate
for feature in range(1, 19):  # Traverse feature numbers X1-X18
    for year in range(1, 6):  # Calculate the year-on-year growth rate of last1-5year in a loop
        new_column = f'X{feature}_last{year}year_ycr'  # New columns for year-on-year growth rates
        
        # If the new column does not exist, create it and fill it with NaN
        if new_column not in modified_df.columns:
            modified_df[new_column] = pd.Series(dtype=float)
            modified_df[new_column].fillna(pd.NA, inplace=True)

        current_column = f'X{feature}_last{year}year'  # Column name for current year
        previous_column = f'X{feature}_last{year+1}year'  # Column name for the Previous year

        # Create temporary DataFrame for calculations without modifying the original DataFrame
        temp_df = modified_df.copy()
        
        # Apply modifications only when the previous_column is 0 (denominator is 0)
        denominator_zero_mask = (temp_df[previous_column] == 0)
        temp_df[previous_column] = temp_df[previous_column].where(~denominator_zero_mask, 0.1)
        
        # Apply modifications only when the current_column is 0
        numerator_zero_mask = (temp_df[current_column] == 0)
        temp_df[current_column] = temp_df[current_column].where(~numerator_zero_mask, 0.1)
        
        # Apply modifications only when both the numerator and the denominator are zero
        both_zero_mask = numerator_zero_mask & denominator_zero_mask
        temp_df[new_column] = temp_df[new_column].where(~both_zero_mask, 0.1)
        
        # Calculate year-on-year growth rate
        temp_df[new_column] = (temp_df[current_column] - temp_df[previous_column]) / temp_df[previous_column]
        
        # Handle missing values in the numerator or denominator
        temp_df[new_column] = temp_df[new_column].where(~(temp_df[current_column].isnull() | temp_df[previous_column].isnull()), pd.NA)
        
        # Update the modified DataFrame with the calculated values
        modified_df[new_column] = temp_df[new_column]

        
# Set pandas display options so that all rows and columns are displayed
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# Select the columns of interest from modified_df
columns_of_interest = [f'X{feature}_last{year}year_ycr' for feature in range(1, 19) for year in range(1, 6)]

# Add the selected columns from modified_df to new_df_transformed
new_df_transformed[columns_of_interest] = modified_df[columns_of_interest]

# Show modified data set
print(new_df_transformed.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X1_last4year  X1_last5year  X1_last6year  \
0         888.5         873.1         954.1      1116.900        1033.7   
1         900.2        1077.4        1008.2       942.700         853.0   
2       13454.0       13582.0        7726.0      5807.000           NaN   
3      353541.0     1037047.0      672072.0       692.991      671429.0   
4           NaN           NaN           NaN           NaN           NaN   

   X2_last1year  X2_last2year  X2_last3year  X2_last4year  X2_last5year  \
0       152

In [10]:
new_df_transformed.shape


(8971, 203)

In [11]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_transformed.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 4978
Number of rows with missing values： 3993


In [12]:
#Adjust the order of column names

import pandas as pd

# Construct a new list of column names, arranged in the required order
new_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']
for year in range(1, 7):  # Loop through the data columns for each year
    for feature in range(1, 19):  # Iterate over the data columns for each feature
        new_columns.append(f'X{feature}_last{year}year')  # Add feature column for current year

for year in range(1, 6):  # Iterate through the year-on-year change rate column
    for feature in range(1, 19):  # Iterate over the year-over-year rate of change column for each feature
        new_columns.append(f'X{feature}_last{year}year_ycr')  # Add the year-over-year rate of change column for the current year

# Reorder the dataset
new_df_transformed = new_df_transformed.reindex(columns=new_columns)

# Display the reordered dataset
print(new_df_transformed.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X2_last1year  X3_last1year  X4_last1year  X5_last1year  X6_last1year  \
0       1524.70       1413.20         177.2          40.5          86.0   
1       1474.50        677.20         650.8          61.5          59.9   
2      21401.00      19334.00          23.0        1686.0       -4407.0   
3    1288165.00        267.81         300.0       46338.0       -5522.0   
4         42.21      79567.00         591.0        2024.0        1849.0   

   X7_last1year  X8_last1year  X9_last1year  X10_last1year  X11_last1year  \
0        

In [13]:
new_df_transformed.shape 

(8971, 203)

In [14]:
#Write DataFrame to Excel file
new_df_transformed.to_excel("company_6y+5ycr_0.1.xlsx", index=False)


## 3. Add stock exchange data

In [15]:
#NYSE

In [16]:
import pandas as pd
file = "NYSE COMPOSITE (DJ) (^NYA) monthly 1999-2018.csv"
nyse_df = pd.read_csv(file)
nyse_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-01-01,6282.060059,6461.180176,6144.080078,6348.890137,6348.890137,0
1,1999-02-01,6317.379883,6346.669922,6097.020020,6201.069824,6201.069824,0
2,1999-03-01,6190.180176,6530.439941,6159.729980,6382.200195,6382.200195,0
3,1999-04-01,6397.000000,6801.759766,6397.000000,6706.919922,6706.919922,0
4,1999-05-01,6811.390137,6887.200195,6492.479980,6579.609863,6579.609863,0


In [17]:
import pandas as pd


# 1. Convert date column to datetime type and extract year information
nyse_df['Date'] = pd.to_datetime(nyse_df['Date'])
nyse_df['Year'] = nyse_df['Date'].dt.year

# 2. Group data by year
grouped = nyse_df.groupby('Year')

# 3. Accumulate and average the data for each year to obtain the mean value of Adj Close for each year.
yearly_avg_adj_close = grouped['Adj Close'].sum() / grouped['Adj Close'].count()

# 4. Add year and mean to a new DataFrame
nyse_year_df = pd.DataFrame({'Year': yearly_avg_adj_close.index, 'Yearly_Avg_Adj_Close': yearly_avg_adj_close.values})

# Print results
print(nyse_year_df)


    Year  Yearly_Avg_Adj_Close
0   1999           6549.827474
1   2000           6805.633260
2   2001           6387.000000
3   2002           5559.102498
4   2003           5478.855835
5   2004           6645.525065
6   2005           7364.758301
7   2006           8434.441610
8   2007           9685.001790
9   2008           8001.502441
10  2009           6100.795776
11  2010           7166.229940
12  2011           7947.905843
13  2012           8049.892497
14  2013           9467.185872
15  2014          10699.956624
16  2015          10606.906738
17  2016          10451.377523
18  2017          11912.848307
19  2018          12593.500651


In [18]:
#NASDAQ

In [19]:
import pandas as pd
file = "NASDAQ Composite (^IXIC) monthly 1999-2018.csv"
nasdaq_df = pd.read_csv(file)
nasdaq_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-01-01,2207.540039,2506.679932,2192.679932,2505.889893,2505.889893,20526060000
1,1999-02-01,2522.379883,2533.439941,2224.209961,2288.030029,2288.030029,17439250000
2,1999-03-01,2286.830078,2520.629883,2235.189941,2461.399902,2461.399902,21196320000
3,1999-04-01,2493.070068,2677.760010,2329.870117,2542.860107,2542.860107,23426400000
4,1999-05-01,2546.330078,2632.739990,2339.120117,2470.520020,2470.520020,18519390000


In [20]:
import pandas as pd

# 1. Convert date column to datetime type and extract year information
nasdaq_df['Date'] = pd.to_datetime(nasdaq_df['Date'])
nasdaq_df['Year'] = nasdaq_df['Date'].dt.year

# 2. Group data by year
grouped = nasdaq_df.groupby('Year')

# 3. Accumulate and average the data for each year to obtain the mean value of Adj Close for each year.
yearly_avg_adj_close = grouped['Adj Close'].sum() / grouped['Adj Close'].count()

# 4. Add year and mean to a new DataFrame
nasdaq_year_df = pd.DataFrame({'Year': yearly_avg_adj_close.index, 'Yearly_Avg_Adj_Close': yearly_avg_adj_close.values})

# Print results
print(nasdaq_year_df)


    Year  Yearly_Avg_Adj_Close
0   1999           2787.559998
1   2000           3710.149170
2   2001           2004.552510
3   2002           1519.770833
4   2003           1659.239176
5   2004           1992.867482
6   2005           2100.603343
7   2006           2278.996664
8   2007           2587.587504
9   2008           2148.948334
10  2009           1856.529999
11  2010           2333.908345
12  2011           2700.576660
13  2012           2984.228353
14  2013           3575.141642
15  2014           4414.850057
16  2015           4932.729126
17  2016           5015.926717
18  2017           6293.024211
19  2018           7405.502482


In [21]:
# Add the annual market closing average price of the two exchanges to the data set

In [22]:
import pandas as pd

# Loop through each year in the last_year column
for i in range(1, 6):  # From 1 to 5
    # Calculate the years to match
    matching_year = new_df_transformed['last_year'] - i + 1
    
    # Add the matched Yearly_Avg_Adj_Close value to new_df_transformed. If the corresponding value cannot be found, fill in the missing value.
    new_df_transformed[f'nyse_last{i}year'] = matching_year.map(nyse_year_df.set_index('Year')['Yearly_Avg_Adj_Close']).fillna(pd.NA)
    new_df_transformed[f'nasdaq_last{i}year'] = matching_year.map(nasdaq_year_df.set_index('Year')['Yearly_Avg_Adj_Close']).fillna(pd.NA)

new_df_transformed.head()


,company_name,status_label,Division,MajorGroup,last_year,X1_last1year,X2_last1year,X3_last1year,X4_last1year,X5_last1year,X6_last1year,X7_last1year,X8_last1year,X9_last1year,X10_last1year,X11_last1year,X12_last1year,X13_last1year,X14_last1year,X15_last1year,X16_last1year,X17_last1year,X18_last1year,X1_last2year,X2_last2year,X3_last2year,X4_last2year,X5_last2year,X6_last2year,X7_last2year,X8_last2year,X9_last2year,X10_last2year,X11_last2year,X12_last2year,X13_last2year,X14_last2year,X15_last2year,X16_last2year,X17_last2year,X18_last2year,X1_last3year,X2_last3year,X3_last3year,X4_last3year,X5_last3year,X6_last3year,X7_last3year,X8_last3year,X9_last3year,X10_last3year,X11_last3year,X12_last3year,X13_last3year,X14_last3year,X15_last3year,X16_last3year,X17_last3year,X18_last3year,X1_last4year,X2_last4year,X3_last4year,X4_last4year,X5_last4year,X6_last4year,X7_last4year,X8_last4year,X9_last4year,X10_last4year,X11_last4year,X12_last4year,X13_last4year,X14_last4year,X15_last4year,X16_last4year,X17_last4year,X18_last4year,X1_last5year,X2_last5year,X3_last5year,X4_last5year,X5_last5year,X6_last5year,X7_last5year,X8_last5year,X9_last5year,X10_last5year,X11_last5year,X12_last5year,X13_last5year,X14_last5year,X15_last5year,X16_last5year,X17_last5year,X18_last5year,X1_last6year,X2_last6year,X3_last6year,X4_last6year,X5_last6year,X6_last6year,X7_last6year,X8_last6year,X9_last6year,X10_last6year,X11_last6year,X12_last6year,X13_last6year,X14_last6year,X15_last6year,X16_last6year,X17_last6year,X18_last6year,X1_last1year_ycr,X2_last1year_ycr,X3_last1year_ycr,X4_last1year_ycr,X5_last1year_ycr,X6_last1year_ycr,X7_last1year_ycr,X8_last1year_ycr,X9_last1year_ycr,X10_last1year_ycr,X11_last1year_ycr,X12_last1year_ycr,X13_last1year_ycr,X14_last1year_ycr,X15_last1year_ycr,X16_last1year_ycr,X17_last1year_ycr,X18_last1year_ycr,X1_last2year_ycr,X2_last2year_ycr,X3_last2year_ycr,X4_last2year_ycr,X5_last2year_ycr,X6_last2year_ycr,X7_last2year_ycr,X8_last2year_ycr,X9_last2year_ycr,X10_last2year_ycr,X11_last2year_ycr,X12_last2year_ycr,X13_last2year_ycr,X14_last2year_ycr,X15_last2year_ycr,X16_last2year_ycr,X17_last2year_ycr,X18_last2year_ycr,X1_last3year_ycr,X2_last3year_ycr,X3_last3year_ycr,X4_last3year_ycr,X5_last3year_ycr,X6_last3year_ycr,X7_last3year_ycr,X8_last3year_ycr,X9_last3year_ycr,X10_last3year_ycr,X11_last3year_ycr,X12_last3year_ycr,X13_last3year_ycr,X14_last3year_ycr,X15_last3year_ycr,X16_last3year_ycr,X17_last3year_ycr,X18_last3year_ycr,X1_last4year_ycr,X2_last4year_ycr,X3_last4year_ycr,X4_last4year_ycr,X5_last4year_ycr,X6_last4year_ycr,X7_last4year_ycr,X8_last4year_ycr,X9_last4year_ycr,X10_last4year_ycr,X11_last4year_ycr,X12_last4year_ycr,X13_last4year_ycr,X14_last4year_ycr,X15_last4year_ycr,X16_last4year_ycr,X17_last4year_ycr,X18_last4year_ycr,X1_last5year_ycr,X2_last5year_ycr,X3_last5year_ycr,X4_last5year_ycr,X5_last5year_ycr,X6_last5year_ycr,X7_last5year_ycr,X8_last5year_ycr,X9_last5year_ycr,X10_last5year_ycr,X11_last5year_ycr,X12_last5year_ycr,X13_last5year_ycr,X14_last5year_ycr,X15_last5year_ycr,X16_last5year_ycr,X17_last5year_ycr,X18_last5year_ycr,nyse_last1year,nasdaq_last1year,nyse_last2year,nasdaq_last2year,nyse_last3year,nasdaq_last3year,nyse_last4year,nasdaq_last4year,nyse_last5year,nasdaq_last5year
0,C_1,alive,D,37,2017.0,942.7,1524.70,1413.20,177.2,40.5,86.0,126.5,335.1,547.90,333.3,15.6,701.2,203.00,1748.30,1.551458e+06,588.4,1748.30,1621.800,888.5,1504.1,1422.700,155.3,71.000,77.2,148.2,344.900,601.1,335.100,56.5,688.000,251.400,1767.60,1200.3288,589.90,1767.60,1619.4,873.1,1442.100,1354.9,136.1,70.800,66.1,136.9,307.70,563.7,329.0,47.7,637.2,242.7,1662.6,842.5112,576.3,1662.6,1525.7,954.1,1515.000,1342.700,85.0,92.300,-8.6,83.7,251.6,566.7,412.0,10.2,603.9,231.1,1594.30,1046.3954,669.90,1594.30,1510.6,1116.900,2199.5,1581.4,564.300,113.40,142.6,256.000,453.60,632.9,402.1,72.9,616.7,297.9,2035.0,9.613080e+05,1198.80,2035.0,1779.0,1033.7,2136.9,1714.5,622.200,108.6,136.600,245.2,452.60,582.9,389.0,55.0,542.4,315.4,2167.1,7.900029e+0

In [23]:
new_df_transformed.shape

(8971, 213)

In [54]:
#Write DataFrame to Excel file
new_df_transformed.to_excel("company_6y+5ycr_0.1_stock.xlsx", index=False)


## 4. Generate new sub-datasets of the dataset, predicting 1-5 years before bankruptcy respectively


#### 4.1 Only data for the last year is generated, excluding the penultimate year.

In [24]:
import pandas as pd

# Select the required columns
selected_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']

# Add last1year column for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last1year'])

# Add last1year_ycr column for each feature
for feature in range(1, 19):
    selected_columns.append(f'X{feature}_last1year_ycr')
    
# Add the required nyse_last and nasdaq_last columns
selected_columns.extend(['nyse_last1year', 'nasdaq_last1year'])

# Generate new data set
new_df_selected1_onlylastyear = new_df_transformed[selected_columns]

# Display the first few rows of the new data set
print(new_df_selected1_onlylastyear.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X2_last1year  X3_last1year  X4_last1year  X5_last1year  X6_last1year  \
0       1524.70       1413.20         177.2          40.5          86.0   
1       1474.50        677.20         650.8          61.5          59.9   
2      21401.00      19334.00          23.0        1686.0       -4407.0   
3    1288165.00        267.81         300.0       46338.0       -5522.0   
4         42.21      79567.00         591.0        2024.0        1849.0   

   X7_last1year  X8_last1year  X9_last1year  X10_last1year  X11_last1year  \
0        

In [25]:
new_df_selected1_onlylastyear.shape

(8971, 43)

In [26]:
# Count the number of missing values in each column
missing_values_count = new_df_selected1_onlylastyear.isnull().sum()

# Show the number of missing values in each column
print("Number of missing values per column")
print(missing_values_count)


Number of missing values per column
company_name           0
status_label           0
Division               0
MajorGroup             0
last_year              0
X1_last1year           0
X2_last1year           0
X3_last1year           0
X4_last1year           0
X5_last1year           0
X6_last1year           0
X7_last1year           0
X8_last1year           0
X9_last1year           0
X10_last1year          0
X11_last1year          0
X12_last1year          0
X13_last1year          0
X14_last1year          0
X15_last1year          0
X16_last1year          0
X17_last1year          0
X18_last1year          0
X1_last1year_ycr     893
X2_last1year_ycr     893
X3_last1year_ycr     893
X4_last1year_ycr     893
X5_last1year_ycr     893
X6_last1year_ycr     893
X7_last1year_ycr     893
X8_last1year_ycr     893
X9_last1year_ycr     893
X10_last1year_ycr    893
X11_last1year_ycr    893
X12_last1year_ycr    893
X13_last1year_ycr    893
X14_last1year_ycr    893
X15_last1year_ycr    893
X16_last1year_

In [27]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_selected1_onlylastyear.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 8078
Number of rows with missing values： 893


In [28]:
# Filter out rows with missing values
rows_with_missing_values =new_df_selected1_onlylastyear[new_df_selected1_onlylastyear.isnull().any(axis=1)]

# Filter out rows with status_label column "failed" among rows with missing values and count their number
failed_rows_with_missing_values = rows_with_missing_values[rows_with_missing_values['status_label'] == 'failed']
count_failed_rows_with_missing_values = len(failed_rows_with_missing_values)

# Displays the number of rows with missing values where status_label is "failed"
print("Number of rows with missing values where status_label is failed：", count_failed_rows_with_missing_values)

# Filter out rows that do not have missing values
rows_without_missing_values = new_df_selected1_onlylastyear[~new_df_selected1_onlylastyear.isnull().any(axis=1)]

# Filter out rows whose status_label column is "failed" among rows that do not have missing values, and count their number
failed_rows_without_missing_values = rows_without_missing_values[rows_without_missing_values['status_label'] == 'failed']
count_failed_rows_without_missing_values = len(failed_rows_without_missing_values)

# Displays the number of rows with status_label "failed" that do not have missing values
print("Number of rows with status_label failed that do not have missing values：", count_failed_rows_without_missing_values)


Number of rows with missing values where status_label is failed： 15
Number of rows with status_label failed that do not have missing values： 594


In [29]:
#Save this subdataset
new_df_selected1_onlylastyear.to_csv("new_df_selected1_onlylastyear_adjusted.csv", index=False)


#### 4.2 Generate a sub-dataset containing the last 2 years

In [30]:
import pandas as pd

# Select the required columns
selected_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']

# Add last1year & last2year columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year' for year in range(1, 3)])

# Add last1year_ycr & last2year_ycr columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year_ycr' for year in range(1, 3)])
    
# Add the required nyse_last and nasdaq_last columns
selected_columns.extend(['nyse_last1year', 'nyse_last2year','nasdaq_last1year','nasdaq_last2year'])

# Generate new data set
new_df_selected2_last2years = new_df_transformed[selected_columns]

# Display the first few rows of the new data set
print(new_df_selected2_last2years.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X2_last1year  X2_last2year  X3_last1year  X3_last2year  \
0         888.5       1524.70        1504.1       1413.20      1422.700   
1         900.2       1474.50        1343.6        677.20       600.500   
2       13454.0      21401.00       27171.0      19334.00        17.589   
3      353541.0    1288165.00      927239.0        267.81    229115.000   
4           NaN         42.21           NaN      79567.00           NaN   

   X4_last1year  X4_last2year  X5_last1year  X5_last2year  X6_last1year  \
0         1

In [31]:
new_df_selected2_last2years.shape

(8971, 81)

In [32]:
# Count the number of missing values in each column
missing_values_count = new_df_selected2_last2years.isnull().sum()

# Show the number of missing values in each column
print("Number of missing values per column")
print(missing_values_count)


Number of missing values per column
company_name            0
status_label            0
Division                0
MajorGroup              0
last_year               0
X1_last1year            0
X1_last2year          893
X2_last1year            0
X2_last2year          893
X3_last1year            0
X3_last2year          893
X4_last1year            0
X4_last2year          893
X5_last1year            0
X5_last2year          893
X6_last1year            0
X6_last2year          893
X7_last1year            0
X7_last2year          893
X8_last1year            0
X8_last2year          893
X9_last1year            0
X9_last2year          893
X10_last1year           0
X10_last2year         893
X11_last1year           0
X11_last2year         893
X12_last1year           0
X12_last2year         893
X13_last1year           0
X13_last2year         893
X14_last1year           0
X14_last2year         893
X15_last1year           0
X15_last2year         893
X16_last1year           0
X16_last2year         893
X1

In [33]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_selected2_last2years.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 7101
Number of rows with missing values： 1870


In [34]:
# Filter out rows with missing values
rows_with_missing_values = new_df_selected2_last2years[new_df_selected2_last2years.isnull().any(axis=1)]

# Filter out rows with status_label column "failed" among rows with missing values and count their number
failed_rows_with_missing_values = rows_with_missing_values[rows_with_missing_values['status_label'] == 'failed']
count_failed_rows_with_missing_values = len(failed_rows_with_missing_values)

# Displays the number of rows with missing values where status_label is "failed"
print("Number of rows with missing values where status_label is failed：", count_failed_rows_with_missing_values)

# Filter out rows that do not have missing values
rows_without_missing_values = new_df_selected2_last2years[~new_df_selected2_last2years.isnull().any(axis=1)]

# Filter out rows whose status_label column is "failed" among rows that do not have missing values, and count their number
failed_rows_without_missing_values = rows_without_missing_values[rows_without_missing_values['status_label'] == 'failed']
count_failed_rows_without_missing_values = len(failed_rows_without_missing_values)

# Displays the number of rows with status_label "failed" that do not have missing values
print("Number of rows with status_label failed that do not have missing values：", count_failed_rows_without_missing_values)


Number of rows with missing values where status_label is failed： 45
Number of rows with status_label failed that do not have missing values： 564


In [35]:
#Save this subdataset
new_df_selected2_last2years.to_csv("new_df_selected2_last2years_adjusted.csv", index=False)


#### 4.3 Generate a sub-dataset containing the last 3 years

In [36]:
import pandas as pd

# Select the required columns
selected_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']

# Add last1year to last3year columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year' for year in range(1, 4)])

# Add last1year_ycr to last3year_ycr columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year_ycr' for year in range(1, 4)])
    
# Add the required nyse_last and nasdaq_last columns
selected_columns.extend(['nyse_last1year', 'nyse_last2year','nyse_last3year','nasdaq_last1year','nasdaq_last2year','nasdaq_last3year'])

# Generate new data set
new_df_selected3_last3years = new_df_transformed[selected_columns]

# Display the first few rows of the new data set
print(new_df_selected3_last3years.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X2_last1year  X2_last2year  X2_last3year  \
0         888.5         873.1       1524.70        1504.1      1442.100   
1         900.2        1077.4       1474.50        1343.6      1921.000   
2       13454.0       13582.0      21401.00       27171.0     14341.000   
3      353541.0     1037047.0    1288165.00      927239.0      1623.383   
4           NaN           NaN         42.21           NaN           NaN   

   X3_last1year  X3_last2year  X3_last3year  X4_last1year  X4_last2year  \
0       141

In [37]:
new_df_selected3_last3years.shape

(8971, 119)

In [38]:
# Count the number of missing values in each column
missing_values_count = new_df_selected3_last3years.isnull().sum()

# Show the number of missing values in each column
print("Number of missing values per column")
print(missing_values_count)


Number of missing values per column
company_name            0
status_label            0
Division                0
MajorGroup              0
last_year               0
X1_last1year            0
X1_last2year          893
X1_last3year         1799
X2_last1year            0
X2_last2year          893
X2_last3year         1799
X3_last1year            0
X3_last2year          893
X3_last3year         1799
X4_last1year            0
X4_last2year          893
X4_last3year         1799
X5_last1year            0
X5_last2year          893
X5_last3year         1799
X6_last1year            0
X6_last2year          893
X6_last3year         1799
X7_last1year            0
X7_last2year          893
X7_last3year         1799
X8_last1year            0
X8_last2year          893
X8_last3year         1799
X9_last1year            0
X9_last2year          893
X9_last3year         1799
X10_last1year           0
X10_last2year         893
X10_last3year        1799
X11_last1year           0
X11_last2year         893
X1

In [39]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_selected3_last3years.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 6298
Number of rows with missing values： 2673


In [40]:
# Filter out rows with missing values
rows_with_missing_values = new_df_selected3_last3years[new_df_selected3_last3years.isnull().any(axis=1)]

# Filter out rows with status_label column "failed" among rows with missing values and count their number
failed_rows_with_missing_values = rows_with_missing_values[rows_with_missing_values['status_label'] == 'failed']
count_failed_rows_with_missing_values = len(failed_rows_with_missing_values)

# Displays the number of rows with missing values where status_label is "failed"
print("Number of rows with missing values where status_label is failed：", count_failed_rows_with_missing_values)

# Filter out rows that do not have missing values
rows_without_missing_values = new_df_selected3_last3years[~new_df_selected3_last3years.isnull().any(axis=1)]

# Filter out rows whose status_label column is "failed" among rows that do not have missing values, and count their number
failed_rows_without_missing_values = rows_without_missing_values[rows_without_missing_values['status_label'] == 'failed']
count_failed_rows_without_missing_values = len(failed_rows_without_missing_values)

# Displays the number of rows with status_label "failed" that do not have missing values
print("Number of rows with status_label failed that do not have missing values：", count_failed_rows_without_missing_values)


Number of rows with missing values where status_label is failed： 95
Number of rows with status_label failed that do not have missing values： 514


In [41]:
#Save this subdataset
new_df_selected3_last3years.to_csv("new_df_selected3_last3years_adjusted.csv", index=False)


#### 4.4 Generate a sub-dataset containing the last 4 years

In [42]:
import pandas as pd

# Select the required columns
selected_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']

# Add last1year to last4year columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year' for year in range(1, 5)])

# Add last1year_ycr to last4year_ycr columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year_ycr' for year in range(1, 5)])
    
# Add the required nyse_last and nasdaq_last columns
selected_columns.extend(['nyse_last1year', 'nyse_last2year', 'nyse_last3year', 'nyse_last4year', 'nasdaq_last1year','nasdaq_last2year','nasdaq_last3year','nasdaq_last4year'])

# Generate new data set
new_df_selected4_last4years = new_df_transformed[selected_columns]

# Display the first few rows of the new data set
print(new_df_selected4_last4years.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X1_last4year  X2_last1year  X2_last2year  \
0         888.5         873.1         954.1       1524.70        1504.1   
1         900.2        1077.4        1008.2       1474.50        1343.6   
2       13454.0       13582.0        7726.0      21401.00       27171.0   
3      353541.0     1037047.0      672072.0    1288165.00      927239.0   
4           NaN           NaN           NaN         42.21           NaN   

   X2_last3year  X2_last4year  X3_last1year  X3_last2year  X3_last3year  \
0      1442

In [43]:
new_df_selected4_last4years.shape

(8971, 157)

In [44]:
# Count the number of missing values in each column
missing_values_count = new_df_selected4_last4years.isnull().sum()

# Show the number of missing values in each column
print("Number of missing values per column")
print(missing_values_count)


Number of missing values per column
company_name            0
status_label            0
Division                0
MajorGroup              0
last_year               0
X1_last1year            0
X1_last2year          893
X1_last3year         1799
X1_last4year         2530
X2_last1year            0
X2_last2year          893
X2_last3year         1799
X2_last4year         2530
X3_last1year            0
X3_last2year          893
X3_last3year         1799
X3_last4year         2530
X4_last1year            0
X4_last2year          893
X4_last3year         1799
X4_last4year         2530
X5_last1year            0
X5_last2year          893
X5_last3year         1799
X5_last4year         2530
X6_last1year            0
X6_last2year          893
X6_last3year         1799
X6_last4year         2530
X7_last1year            0
X7_last2year          893
X7_last3year         1799
X7_last4year         2530
X8_last1year            0
X8_last2year          893
X8_last3year         1799
X8_last4year         2530
X9

In [45]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_selected4_last4years.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 5600
Number of rows with missing values： 3371


In [46]:
# Filter out rows with missing values
rows_with_missing_values = new_df_selected4_last4years[new_df_selected4_last4years.isnull().any(axis=1)]

# Filter out rows with status_label column "failed" among rows with missing values and count their number
failed_rows_with_missing_values = rows_with_missing_values[rows_with_missing_values['status_label'] == 'failed']
count_failed_rows_with_missing_values = len(failed_rows_with_missing_values)

# Displays the number of rows with missing values where status_label is "failed"
print("Number of rows with missing values where status_label is failed：", count_failed_rows_with_missing_values)

# Filter out rows that do not have missing values
rows_without_missing_values = new_df_selected4_last4years[~new_df_selected4_last4years.isnull().any(axis=1)]

# Filter out rows whose status_label column is "failed" among rows that do not have missing values, and count their number
failed_rows_without_missing_values = rows_without_missing_values[rows_without_missing_values['status_label'] == 'failed']
count_failed_rows_without_missing_values = len(failed_rows_without_missing_values)

# Displays the number of rows with status_label "failed" that do not have missing values
print("Number of rows with status_label failed that do not have missing values：", count_failed_rows_without_missing_values)


Number of rows with missing values where status_label is failed： 133
Number of rows with status_label failed that do not have missing values： 476


In [47]:
#Save this subdataset
new_df_selected4_last4years.to_csv("new_df_selected4_last4years_adjusted.csv", index=False)


#### 4.5 Generate a sub-dataset containing the last 5 years

In [48]:
import pandas as pd

# Select the required columns
selected_columns = ['company_name', 'status_label', 'Division', 'MajorGroup', 'last_year']

# Add last1year to last5year columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year' for year in range(1, 6)])

# Add last1year_ycr & last5year_ycr columns for each feature
for feature in range(1, 19):
    selected_columns.extend([f'X{feature}_last{year}year_ycr' for year in range(1, 6)])
    
# Add the required nyse_last and nasdaq_last columns
selected_columns.extend(['nyse_last1year', 'nyse_last2year', 'nyse_last3year', 'nyse_last4year', 'nyse_last5year', 'nasdaq_last1year','nasdaq_last2year', 'nasdaq_last3year','nasdaq_last4year','nasdaq_last5year'])

# Generate new data set
new_df_selected5_last5years = new_df_transformed[selected_columns]

# Display the first few rows of the new data set
print(new_df_selected5_last5years.head())


  company_name status_label Division  MajorGroup  last_year  X1_last1year  \
0          C_1        alive        D          37     2017.0         942.7   
1          C_2        alive        D          36     2010.0        1107.7   
2          C_3        alive        D          38     2008.0       12686.0   
3          C_4        alive        D          28     2007.0      581502.0   
4          C_5        alive        D          35     1999.0       28957.0   

   X1_last2year  X1_last3year  X1_last4year  X1_last5year  X2_last1year  \
0         888.5         873.1         954.1      1116.900       1524.70   
1         900.2        1077.4        1008.2       942.700       1474.50   
2       13454.0       13582.0        7726.0      5807.000      21401.00   
3      353541.0     1037047.0      672072.0       692.991    1288165.00   
4           NaN           NaN           NaN           NaN         42.21   

   X2_last2year  X2_last3year  X2_last4year  X2_last5year  X3_last1year  \
0        15

In [49]:
new_df_selected5_last5years.shape

(8971, 195)

In [50]:
# Count the number of missing values in each column
missing_values_count = new_df_selected5_last5years.isnull().sum()

# Show the number of missing values in each column
print("Number of missing values per column")
print(missing_values_count)


Number of missing values per column
company_name            0
status_label            0
Division                0
MajorGroup              0
last_year               0
X1_last1year            0
X1_last2year          893
X1_last3year         1799
X1_last4year         2530
X1_last5year         3191
X2_last1year            0
X2_last2year          893
X2_last3year         1799
X2_last4year         2530
X2_last5year         3191
X3_last1year            0
X3_last2year          893
X3_last3year         1799
X3_last4year         2530
X3_last5year         3191
X4_last1year            0
X4_last2year          893
X4_last3year         1799
X4_last4year         2530
X4_last5year         3191
X5_last1year            0
X5_last2year          893
X5_last3year         1799
X5_last4year         2530
X5_last5year         3191
X6_last1year            0
X6_last2year          893
X6_last3year         1799
X6_last4year         2530
X6_last5year         3191
X7_last1year            0
X7_last2year          893
X7

In [51]:
# Count the number of rows with missing values
rows_with_missing_values = new_df_selected5_last5years.isnull().any(axis=1)
# Group and count rows with and without missing values
missing_values_counts = rows_with_missing_values.value_counts()

# Show the number of rows with and without missing values
print("Number of rows without missing values：", missing_values_counts[False])
print("Number of rows with missing values：", missing_values_counts[True])


Number of rows without missing values： 4978
Number of rows with missing values： 3993


In [52]:
# Filter out rows with missing values
rows_with_missing_values = new_df_selected5_last5years[new_df_selected5_last5years.isnull().any(axis=1)]

# Filter out rows with status_label column "failed" among rows with missing values and count their number
failed_rows_with_missing_values = rows_with_missing_values[rows_with_missing_values['status_label'] == 'failed']
count_failed_rows_with_missing_values = len(failed_rows_with_missing_values)

# Displays the number of rows with missing values where status_label is "failed"
print("Number of rows with missing values where status_label is failed：", count_failed_rows_with_missing_values)

# Filter out rows that do not have missing values
rows_without_missing_values = new_df_selected5_last5years[~new_df_selected5_last5years.isnull().any(axis=1)]

# Filter out rows whose status_label column is "failed" among rows that do not have missing values, and count their number
failed_rows_without_missing_values = rows_without_missing_values[rows_without_missing_values['status_label'] == 'failed']
count_failed_rows_without_missing_values = len(failed_rows_without_missing_values)

# Displays the number of rows with status_label "failed" that do not have missing values
print("Number of rows with status_label failed that do not have missing values：", count_failed_rows_without_missing_values)


Number of rows with missing values where status_label is failed： 179
Number of rows with status_label failed that do not have missing values： 430


In [53]:
#Save this subdataset
new_df_selected5_last5years.to_csv("new_df_selected5_last5years_adjusted.csv", index=False)
